# Building Voice Agents with Azure OpenAI Realtime API
## WeAreDevelopers Conference Workshop (20 minutes)

### 🎯 Workshop Objectives
By the end of this **20-minute session**, you will:
- **See** a live demo of our TPS report voice agent
- **Understand** the core architecture and key components  
- **Get** the complete code to build your own voice agent
- **Know** next steps for production deployment

### 🚀 What We'll Cover (Live Demo Focus)
1. **Quick Setup** (2 min) - Environment and credentials
2. **Live Demo** (12 min) - TPS report voice agent in action
3. **Code Walkthrough** (5 min) - Key implementation highlights
4. **Next Steps** (1 min) - Resources and production tips

### 🎭 The Demo: TPS Reports Voice Agent
We're showcasing a hands-free voice agent that helps employees file TPS reports while driving:
- Asks if the user has filed their TPS report
- Collects report details through natural conversation
- Generates a structured JSON report via function calling
- Handles interruptions and real conversation flow

**Let's see it in action!** 🚀

In [ ]:
!python -m venv .venv
!.venv/Scripts/activate # or .venv/bin/activate on Unix
!pip install -r requirements.txt

**Download Connection Details**

Use this [link](https://pwpush.com/p/godbrgvnmdm/r) to download the content for your `.env` file.  
Copy and paste the downloaded content into your local `.env` file to configure your environment.

## ⚡ Quick Setup (2 minutes)

Before we demo, let's quickly verify our environment and Azure OpenAI credentials.

In [ ]:
# Quick Azure OpenAI credentials check
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Azure OpenAI configuration
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_DEPLOYMENT_NAME = os.getenv("AZURE_VOICE_COMPLETION_DEPLOYMENT_NAME") 
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")

# Quick verification
print("🔧 Azure OpenAI Configuration:")
print(f"📍 Endpoint: {'✅ Configured' if AZURE_OPENAI_ENDPOINT else '❌ Missing'}")
print(f"🚀 Deployment: {'✅ Configured' if AZURE_DEPLOYMENT_NAME else '❌ Missing'}")
print(f"🔑 API Key: {'✅ Configured' if AZURE_OPENAI_API_KEY else '❌ Missing'}")

if AZURE_OPENAI_ENDPOINT and AZURE_DEPLOYMENT_NAME and AZURE_OPENAI_API_KEY:
    print("\n✨ Ready for demo!")
else:
    print("\n⚠️ Please check your .env file configuration")
    print("Required: AZURE_OPENAI_ENDPOINT, AZURE_VOICE_COMPLETION_DEPLOYMENT_NAME, AZURE_OPENAI_API_KEY")

## ?️ Architecture Overview (Demo Context)

Our voice agent has 4 key components working together in real-time:

1. **Frontend (Browser)**: Captures audio, sends to middle tier, plays responses
2. **Middle Tier (Python)**: Routes messages, manages tools, handles auth
3. **Azure OpenAI**: Processes voice, runs conversation, calls functions
4. **Custom Tools**: Business logic (our TPS report generator)

**Real-time Flow:**
```
🎤 User speaks → 🌐 WebSocket → 🧠 Azure OpenAI → 🛠️ Function Call → 📊 JSON Report
```

**Now let's see it working!**

## 🏗️ Voice Agent Architecture Overview

<img src="img/voice_agent_architecture.png" alt="Voice Agent Architecture" style="width: 100%; max-width: 800px; height: auto; display: block; margin: 0 auto; border-radius: 8px; box-shadow: 0 4px 8px rgba(0,0,0,0.1);">

### 🔄 Real-time Data Flow Example:

1. 🎤 **User**: "I need to file TPS report 123 for Acme Corp"
2. 🌐 **WebSocket** sends audio to RTMiddleTier
3. 🔄 **RTMiddleTier** forwards to Azure OpenAI
4. 🧠 **Azure** processes speech & understands intent
5. 🛠️ **Function call** triggered: `generate_report()`
6. 📊 **Tool** generates JSON report
7. 🔊 **Response** sent back to user as speech

## 🚀 Live Demo Setup

Let's quickly set up our voice agent components for the live demo. We'll focus on the key pieces that make the magic happen!

In [ ]:
# Essential classes for our voice agent demo
import json
from enum import Enum
from datetime import datetime

class ToolResultDirection(Enum):
    TO_SERVER = 1    # Send result back to Azure OpenAI
    TO_CLIENT = 2    # Send result to the frontend client

class ToolResult:
    def __init__(self, text: str, destination: ToolResultDirection):
        self.text = text
        self.destination = destination
    
    def to_text(self) -> str:
        return self.text if isinstance(self.text, str) else json.dumps(self.text)

class Tool:
    def __init__(self, target, schema: dict):
        self.target = target    # Function to execute
        self.schema = schema    # JSON schema for the tool

# Our TPS Report Tool - the star of the demo!
async def generate_report_tool(args: dict) -> ToolResult:
    """Generate a TPS report from conversation data"""
    report_data = {
        "tps_report_id": args.get("tps_report_id"),
        "customer_name": args.get("customer_name"), 
        "hours_spent": args.get("hours_spent"),
        "status": args.get("status"),
        "generated_at": datetime.now().isoformat(),
        "generated_by": "Voice Agent Demo"
    }
    
    print(f"📋 Generated TPS Report: {report_data}")
    return ToolResult(report_data, ToolResultDirection.TO_CLIENT)

# Tool schema for Azure OpenAI function calling
tps_report_schema = {
    "type": "function",
    "name": "generate_report",
    "description": "Generates a JSON TPS report from conversation data",
    "parameters": {
        "type": "object",
        "properties": {
            "tps_report_id": {"type": "string", "description": "Three-digit report ID"},
            "customer_name": {"type": "string", "description": "Customer name"}, 
            "hours_spent": {"type": "string", "description": "Hours spent"},
            "status": {"type": "string", "enum": ["active", "done", "postponed"]}
        },
        "required": ["tps_report_id", "customer_name", "hours_spent", "status"]
    }
}

# Create our demo tool
tps_tool = Tool(target=generate_report_tool, schema=tps_report_schema)
print("✅ TPS Report Tool ready for demo!")

## 🛠️ Function Calling Flow & Schema Structure

### Function Calling Process
<img src="img/function_calling.png" alt="Function Calling Flow" style="width: 100%; max-width: 800px; height: auto; display: block; margin: 0 auto; border-radius: 8px; box-shadow: 0 4px 8px rgba(0,0,0,0.1);">

### Tool Schema Structure
<img src="img/tools_schema.png" alt="Tools Schema" style="width: 100%; max-width: 800px; height: auto; display: block; margin: 0 auto; border-radius: 8px; box-shadow: 0 4px 8px rgba(0,0,0,0.1);">

### 🎯 Key Insights:
- **Function calling** bridges natural conversation and structured data
- **Schema** defines exactly what information to extract  
- **Azure OpenAI** automatically maps speech to function parameters
- **Tools** can return data to client, server, or both

## 🎤 Demo: Voice Agent Configuration

Now let's configure our voice agent with the TPS report conversation flow and start the demo!

In [ ]:
# Configure our TPS Report Voice Agent for Demo
from azure.core.credentials import AzureKeyCredential
from azure.identity import DefaultAzureCredential

# Import our RTMiddleTier (this exists in backend/rtmt.py)
from backend.rtmt import RTMiddleTier

# The conversation script for our voice agent
TPS_SYSTEM_MESSAGE = """
You are a helpful assistant for TPS report filing. The user is driving and talking hands-free.

Start by asking: "Have you filed your TPS report today?"

If YES: Congratulate them and wish them a good day.
If NO: Help them file it by collecting:
1. TPS report ID (3 digits)
2. Customer name  
3. Hours spent
4. Status (active/done/postponed)

After collecting all info, call the 'generate_report' function to create their report.
Be conversational and friendly!
"""

# Create and configure the voice agent
print("🎙️ Setting up TPS Report Voice Agent...")

rtmt = RTMiddleTier(
    endpoint=AZURE_OPENAI_ENDPOINT,
    deployment=AZURE_DEPLOYMENT_NAME,
    credentials=AzureKeyCredential(AZURE_OPENAI_API_KEY) if AZURE_OPENAI_API_KEY else DefaultAzureCredential()
)

# Configure the agent
rtmt.system_message = TPS_SYSTEM_MESSAGE
rtmt.tools["generate_report"] = tps_tool
rtmt.temperature = 0.7

print("✅ Voice agent configured!")
print(f"🛠️ Tools: {list(rtmt.tools.keys())}")
print("🎬 Ready for live demo!")

In [ ]:
# 🚀 Launch the Voice Agent Demo!
from aiohttp import web
from pathlib import Path
import asyncio

async def create_demo_app():
    """Create the demo web application"""
    app = web.Application()
    
    # Attach our voice agent
    rtmt.attach_to_app(app, "/realtime")
    
    # Serve the frontend
    static_dir = Path("static")
    app.router.add_get('/', lambda req: web.FileResponse(static_dir / "index.html"))
    app.router.add_static('/static/', path=str(static_dir), name='static')
    
    return app

print("🎬 LIVE DEMO TIME!")
print("=" * 50)
print("🌐 Demo URL: http://localhost:8765")
print("🎙️ WebSocket: ws://localhost:8765/realtime")
print("📋 Expected conversation flow:")
print("   1. Agent asks: 'Have you filed your TPS report?'")
print("   2. User says: 'No, I need to file it'")
print("   3. Agent collects: ID, customer, hours, status")
print("   4. Agent generates JSON report")
print()
print("🎯 Demo conversation example:")
print("   👤 'No, I haven't filed it yet'")
print("   🤖 'What's the report ID?'")
print("   👤 'Report 123'")
print("   🤖 'Which customer?'")
print("   👤 'Acme Corporation'")
print("   🤖 'How many hours?'")
print("   👤 'About 8 hours'")
print("   🤖 'What's the status?'")
print("   👤 'Active'")
print("   🤖 [Generates JSON report]")
print()

# For notebook demo - show setup without actually starting server
print("🔧 Demo Setup Complete!")
print("📝 To run the actual demo server:")
print("1. Run: python app.py")
print("2. Open: http://localhost:8765")
print("3. Allow microphone access")
print("4. Click 'Start Conversation'")

# Optional: For actual demo launch (uncomment if running in production)
async def start_demo():
    app = await create_demo_app()
    runner = web.AppRunner(app)
    await runner.setup()
    site = web.TCPSite(runner, "localhost", 8765)
    await site.start()
    print("✅ Demo running at http://localhost:8765")
    return runner

# To actually start: 
runner = await start_demo()


## 🎭 TPS Report Conversation Flow Demo

<img src="img/conversation_flow.png" alt="TPS Report Conversation Flow" style="width: 100%; max-width: 800px; height: auto; display: block; margin: 0 auto; border-radius: 8px; box-shadow: 0 4px 8px rgba(0,0,0,0.1);">

### 🎬 Demo Highlights:
- **Natural conversation flow** with interruptions and clarifications
- **Automatic data extraction** from unstructured speech  
- **Function calling** triggered when all required data is collected
- **Real-time processing** with < 500ms response time
- **Hands-free operation** perfect for driving scenarios

## 🖥️ Code Walkthrough (5 minutes)

While the demo runs, let's quickly walk through the key code components that make this work:

In [ ]:
# Key Architecture Components

print("🏗️ Voice Agent Architecture Highlights:")
print("=" * 45)

components = {
    "1. Frontend (HTML/JS)": [
        "• Captures microphone audio",
        "• WebSocket connection to /realtime",
        "• Plays audio responses",
        "• Shows generated reports"
    ],
    "2. RTMiddleTier (Python)": [
        "• Routes messages between client & Azure OpenAI",
        "• Manages custom tools (our TPS generator)",
        "• Handles authentication",
        "• Enforces conversation flow"
    ],
    "3. Azure OpenAI Realtime API": [
        "• Speech-to-text conversion",
        "• Natural conversation processing", 
        "• Function calling (tool execution)",
        "• Text-to-speech responses"
    ],
    "4. Custom Tools": [
        "• Business logic (TPS report generation)",
        "• JSON schema for function calling",
        "• Result routing (to client/server)"
    ]
}

for component, details in components.items():
    print(f"\n{component}:")
    for detail in details:
        print(f"  {detail}")

print("\n🔄 Real-time Flow:")
print("🎤 Audio → 🌐 WebSocket → 🧠 AI Processing → 🛠️ Tool Call → 📊 JSON Result")

print("\n✨ The magic happens in real-time conversation!")
print("💡 All code available in this notebook for you to take home!")

## ⚡ Technical Implementation Overview

### 🌐 WebSocket Message Flow
<img src="img/message_flow.png" alt="WebSocket Message Flow" style="width: 100%; max-width: 800px; height: auto; display: block; margin: 0 auto; border-radius: 8px; box-shadow: 0 4px 8px rgba(0,0,0,0.1);">

### 🎵 Audio Processing Pipeline
<img src="img/processing_pipeline.png" alt="Audio Processing Pipeline" style="width: 100%; max-width: 800px; height: auto; display: block; margin: 0 auto; border-radius: 8px; box-shadow: 0 4px 8px rgba(0,0,0,0.1);">

### ⚡ Real-time Performance Metrics
| Metric | Value | Description |
|--------|-------|-------------|
| 🎯 **Latency** | < 500ms | End-to-end response time |
| 🔊 **Audio Quality** | 24kHz | High-fidelity speech |
| 🎙️ **VAD Threshold** | 0.7 | Voice activity detection |
| 📦 **Buffer Size** | 4096 | Audio processing chunks |
| 🌐 **WebSocket** | WSS | Secure real-time connection |

### 🏭 Production Deployment Architecture
<img src="img/production_deployment.png" alt="Production Deployment" style="width: 100%; max-width: 800px; height: auto; display: block; margin: 0 auto; border-radius: 8px; box-shadow: 0 4px 8px rgba(0,0,0,0.1);">

### 🔧 Technical Highlights:
- **WebSocket** enables full-duplex real-time communication
- **Audio processing** pipeline handles format conversions seamlessly
- **Performance** optimized for < 500ms end-to-end latency
- **Production-ready** with Azure services for scale and reliability

In [ ]:
# Key Implementation Highlights
import json

print("🔧 Implementation Deep Dive:")
print("=" * 35)

# 1. Function Calling Schema
print("1️⃣ FUNCTION CALLING MAGIC")
print("Our tool schema tells Azure OpenAI exactly what data to collect:")

# TPS Report Schema (self-contained for this demo)
tps_schema_example = {
    "type": "function",
    "name": "generate_report",
    "description": "Generates a JSON TPS report from conversation data",
    "parameters": {
        "type": "object",
        "properties": {
            "tps_report_id": {"type": "string", "description": "Three-digit report ID"},
            "customer_name": {"type": "string", "description": "Customer name"}, 
            "hours_spent": {"type": "string", "description": "Hours spent"},
            "status": {"type": "string", "enum": ["active", "done", "postponed"]}
        },
        "required": ["tps_report_id", "customer_name", "hours_spent", "status"]
    }
}

print(f"""
{json.dumps(tps_schema_example, indent=2)}
""")

# 2. Real-time Processing
print("2️⃣ REAL-TIME PROCESSING")
print("• WebSocket enables bidirectional real-time communication")
print("• Server-side Voice Activity Detection (VAD)")
print("• 24kHz audio streaming")
print("• Interrupt handling (user can cut off AI)")

# 3. System Message Power
print("3️⃣ CONVERSATION CONTROL")
print("System message defines the entire conversation flow:")
print("• Specific question sequence")
print("• When to call functions")
print("• Response personality")

# 4. Production Considerations
print("4️⃣ PRODUCTION READY FEATURES")
production_features = [
    "Authentication with Azure (API keys or managed identity)",
    "Error handling and retries", 
    "Audio format conversion",
    "WebSocket connection management",
    "Tool result routing (client vs server)",
    "Scalable deployment patterns"
]

for i, feature in enumerate(production_features, 1):
    print(f"  {i}. {feature}")

print("\n✨ This is a complete, production-ready voice agent foundation!")
print("💡 Perfect starting point for enterprise voice applications")

## 🎯 Wrap-up & Next Steps (1 minute)

### 🎉 What You Just Experienced

In 20 minutes, you've seen a complete voice agent in action:
- **Real-time conversation** with Azure OpenAI Realtime API
- **Function calling** to execute business logic
- **WebSocket architecture** for low-latency communication
- **Production-ready patterns** you can use immediately

### 🚀 Immediate Next Steps

1. **? Get the Code**: This entire notebook is yours to take!
2. **? Set up Azure OpenAI**: Get your realtime preview access
3. **🛠️ Customize**: Replace TPS reports with your business logic
4. **🌐 Deploy**: Use Azure Container Apps for production

### ? Resources to Continue

- **Complete code**: Available in this notebook
- **Azure OpenAI Realtime API docs**: [docs.microsoft.com](https://docs.microsoft.com/azure/cognitive-services/openai/)
- **Production deployment guide**: Check the notebook appendix
- **Community samples**: [github.com/azure-samples](https://github.com/azure-samples)

### 🤝 Thank You!

**Questions?** Find me after the session!

**Keep building amazing voice experiences!** 🎙️✨

## 🚀 Voice Agent Use Cases & Learning Journey

### Use Cases & Applications
<img src="img/use_cases.png" alt="Use Cases & Applications" style="width: 100%; max-width: 800px; height: auto; display: block; margin: 0 auto; border-radius: 8px; box-shadow: 0 4px 8px rgba(0,0,0,0.1);">

### 📖 Workshop Resources Summary:

#### 💡 Key Concepts Learned:
- **Real-time WebSocket architecture**
- **Function calling with structured data**
- **Audio processing pipeline**
- **Production deployment patterns**

#### ⚡ Quick Wins:
- **Copy notebook and modify TPS logic**
- **Deploy to Azure Container Apps**
- **Add your own custom functions**
- **Integrate with existing APIs**

### 🎉 You're ready to build amazing voice experiences!
### 🤝 Questions? Find me after the session!